In [1]:
import folium
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [2]:
df = pd.read_csv("Active Hotel Residentials at the End of 2019.xlsx - Report_BL_ByNAICCode_CLLE(6).csv")

In [3]:
(df.Longitude.min() + df.Longitude.max()) / 2, (df.Latitude.min() + df.Latitude.max()) / 2

(-78.48748115000001, 38.03686463)

In [4]:
df = df[(df.Latitude.notnull()) & (df.Longitude.notnull())]
df

,Trading As Name,Address as given,Zip,Address,Latitude,Longitude,Zoning
0,Morrill Whitney W,1104 Locust Ave 00000,22901,1104 Locust Ave,38.042926,-78.460015,R-1S
1,Dettra Anna,624 WATSON AVE 22901,22901,624 WATSON AVE,38.039816,-78.469584,R-1
2,Rohdie Matthew & Downey Jennifer,1014 LOCUST LN 22901,22901,1014 LOCUST LN,38.044259,-78.456346,R-1S
3,Tiny on Belleview,1212 BELLEVIEW AVE 22901,22901,1212 BELLEVIEW AVE,38.038333,-78.461305,R-2
4,Oliva Cary,1012 LOCUST LN 22901,22901,1012 LOCUST LN,38.044184,-78.456746,R-1S
5,Monteagudo Maria C,1419 LESTER DR 22901,22901,1419 LESTER DR,38.058290,-78.473782,R-1
6,Johne Arthur,1215 MONTICELLO RD 22901,22901,1215 MONTICELLO RD,38.020706,-78.473125,B-3
7,Claasen Johanna A,1010 LOCUST LN 22901,22901,1010 LOCUST LN,38.044300,-78.456867,R-1S
8,Price White,1012 LOCUST AVE 22901,22901,1012 LOCUST AVE,38.040778,-78.462472,R-1S
9,Redmont Holdings LLC,603 WATSON AVE 22901,22901,603 WATSON AVE,38.040903,-78.470711,R-1H


In [5]:
# geometry = [Point(xy) for xy in zip(df.Longitude, df.Latitude)]
# df = df.drop(['Longitude', 'Latitude'], axis=1)
# crs = {'init': 'epsg:4326'}
# gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
# gdf

In [6]:
folium_map = folium.Map(location=[38.03686463, -78.48748115000001],
                        zoom_start=14,
                        tiles="CartoDB positron")

for idx, zone in enumerate(list(df.Zoning.unique())):
    fg=folium.FeatureGroup(name="Zone: {}".format(zone))
    tmp = df[df.Zoning == zone]
    
    for idx, row in tmp.iterrows():
        popup_text = "Name: {}<br> Address: {}<br> Zoning: {}"
        popup_text = popup_text.format(row["Trading As Name"],row["Address"],row["Zoning"])
        fg.add_child(folium.CircleMarker(location=(row["Latitude"], row["Longitude"]),popup=popup_text,fill=True))
     #   cm.add_to(folium_map)

    folium_map.add_child(fg)
    

lc = folium.LayerControl().add_to(folium_map)

folium_map

In [7]:
folium_map.save("my_map.html")